In [ ]:
import os, glob, re, pickle
from functools import partial
from collections import OrderedDict
import operator as op
from cytoolz import compose

import pandas as pd
import seaborn as sns
import numpy as np
import scanpy as sc
import anndata as ad
import matplotlib as mpl
import matplotlib.pyplot as plt

from pyscenic.export import export2loom, add_scenic_metadata
from pyscenic.utils import load_motifs, modules_from_adjacencies
from pyscenic.transform import df2regulons
from pyscenic.aucell import aucell
from pyscenic.binarization import binarize
from pyscenic.rss import regulon_specificity_scores
from pyscenic.plotting import plot_binarization, plot_rss
from ctxcore.rnkdb import FeatherRankingDatabase as RankingDatabase
from pyscenic.prune import prune2df, df2regulons

from IPython.display import HTML, display
from dask.distributed import Client, LocalCluster

In [1]:
from arboreto.algo import grnboost2
import os
import numpy as np
import pandas as pd
import scanpy as sc
import loompy as lp
import json
import base64
import zlib
from pyscenic.plotting import plot_binarization
from pyscenic.export import add_scenic_metadata
from pyscenic.cli.utils import load_signatures
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
!pwd

/groups/mb928_gp/mp4486/Pyscenic


In [4]:
#Read in anndata object
adata = sc.read_h5ad("../scgpt/data/rna_leidein.h5ad")

In [5]:
# Calculate how many cells each gene is expressed in
gene_cell_counts = np.array((adata.X > 0).sum(axis=0)).flatten()

# Define threshold: 0.1% of total cells
min_cells = int(0.001 * adata.n_obs)

# Identify genes expressed in at least min_cells
genes_kept_mask = gene_cell_counts >= min_cells
genes_kept = adata.var_names[genes_kept_mask]

# Subset the AnnData object
adata_filtered = adata[:, genes_kept].copy()
adata_filtered

AnnData object with n_obs × n_vars = 495037 × 27510
    obs: 'orig.ident', 'nCount_ATAC', 'nFeature_ATAC', 'nCount_RNA', 'nFeature_RNA', 'mitoPercent', 'gex_barcode', 'gex_raw_reads', 'atac_barcode', 'atac_raw_reads', 'atac_unmapped_reads', 'atac_mitochondrial_reads', 'atac_fragments', 'atac_TSS_fragments', 'atac_peak_region_fragments', 'atac_peak_region_cutsites', 'scDblFinder.class', 'scDblFinder.score', 'scDblFinder.weighted', 'scDblFinder.cxds_score', 'RNA_snn_res.0.8', 'seurat_clusters', 'sample', 'orig_sample_cluster', 'seurat_clusters_12162024', 'scDblFinder.score_12162024', 'RNA_snn_res.1.6', 'ident', 'batch', 'ID4', 'Brain_bank', 'seq_study_ID', 'Sex', 'Age', 'PMI', 'Diagnosis', 'Suicide', 'RIN', 'sample_barcode', 'harmony_rna_clusters_0.5', 'seurat_rna_clusters_0.5', 'harmony_rna_clusters_0.25', 'n_cells', 'predictions', 'cell_group', 'nucleosome_signal', 'nucleosome_percentile', 'TSS.enrichment', 'TSS.percentile', 'pct_reads_in_peaks', 'blacklist_fraction', 'atac_clusters_0.

In [4]:
!python3 --version

Python 3.12.6


In [6]:
# Set path names as variables
BASE_PATH = "resources"

MOTIFS_PATH = os.path.join(BASE_PATH, 'motifs-v9-nr.hgnc-m0.001-o0.0.tbl')
CURATED_TFS_PATH = os.path.join(BASE_PATH, 'tf_names.txt')

# OUT_TFS_ALL_PATH = os.path.join(BASE_PATH, 'all_tfs.txt')
# OUT_TFS_PATH = os.path.join(BASE_PATH, 'overlapping_tfs.txt')
# OUT_MODULES_PATH = os.path.join(BASE_PATH, 'modules.p')
# OUT_MOTIFS_PATH = os.path.join(BASE_PATH, 'motifs.csv')
# OUT_REGULONS_PATH = os.path.join(BASE_PATH, 'regulons.p')
# OUT_AUC_MTX_PATH = os.path.join(BASE_PATH, 'auc_mtx.csv')

In [7]:
# #Read in motif annotations and get all the unique TFs with a motif annotation
motifs_df = pd.read_csv(MOTIFS_PATH, sep='\t')
tfs = motifs_df.gene_name.unique()
len(tfs)

/tmp/mp4486/ipykernel_2709926/3569383860.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  motifs_df = pd.read_csv(MOTIFS_PATH, sep='\t')


1839

In [8]:
!pwd

/groups/mb928_gp/mp4486/Pyscenic


In [9]:
# #Read in the curated TFs file from Lambert et al
with open(CURATED_TFS_PATH, 'rt') as f:
    curated_tfs = list(map(lambda s: s.strip(), f.readlines()))

len(curated_tfs)

1639

In [10]:
with open('resources/sig_tfs_from_TOBIAS.txt', 'r') as f:
    tobias_tfs = [line.strip() for line in f]

len(tobias_tfs)

232

In [25]:
# #Find the overlap with the curated human TFs
overlapping_tfs = list(set(tfs).intersection(curated_tfs))
len(overlapping_tfs)

1390

## Dask

In [23]:
client.close()
cluster.close()

In [12]:
!nproc

128


In [13]:
from dask.distributed import Client, LocalCluster

# System resources
total_cores = 128
total_memory_gb = 700

# Set Dask configuration
n_workers = 32                # Number of processes (not threads)
threads_per_worker = 4       # Number of threads in each process (4x4 = 16 cores total)
memory_per_worker = "18GB"  # 600GB / 4 workers

# Create Dask cluster
cluster = LocalCluster(
    n_workers=n_workers,
    threads_per_worker=threads_per_worker,
    memory_limit=memory_per_worker,
)

client = Client(cluster)
print("Dask dashboard:", client.dashboard_link)

Dask dashboard: http://127.0.0.1:8787/status


## SUBSET TO JUST Neurogenic Clusters


In [14]:
traj_cells = pd.read_csv('just_trajectory_cells_12.19.25.csv')
keep_barcodes = list(traj_cells['x'])

In [15]:
adata.obs['sample_barcode']

CM021_AAACAGCCAAACCTTG-1    CM021_AAACAGCCAAACCTTG-1
CM021_AAACAGCCAAGCCAGA-1    CM021_AAACAGCCAAGCCAGA-1
CM021_AAACAGCCACCTGCCT-1    CM021_AAACAGCCACCTGCCT-1
CM021_AAACAGCCAGCTCATA-1    CM021_AAACAGCCAGCTCATA-1
CM021_AAACCAACACACAATT-1    CM021_AAACCAACACACAATT-1
                                      ...           
MM152_TTTGGCTGTTAGTGAT-1    MM152_TTTGGCTGTTAGTGAT-1
MM152_TTTGGCTGTTGCGCGA-1    MM152_TTTGGCTGTTGCGCGA-1
MM152_TTTGTCTAGGCATGTT-1    MM152_TTTGTCTAGGCATGTT-1
MM152_TTTGTGAAGGCGCACT-1    MM152_TTTGTGAAGGCGCACT-1
MM152_TTTGTGGCACCCACCT-1    MM152_TTTGTGGCACCCACCT-1
Name: sample_barcode, Length: 495037, dtype: object

In [16]:
adata_subset = adata_filtered[adata_filtered.obs['sample_barcode'].isin(keep_barcodes)].copy()
adata_subset

AnnData object with n_obs × n_vars = 18941 × 27510
    obs: 'orig.ident', 'nCount_ATAC', 'nFeature_ATAC', 'nCount_RNA', 'nFeature_RNA', 'mitoPercent', 'gex_barcode', 'gex_raw_reads', 'atac_barcode', 'atac_raw_reads', 'atac_unmapped_reads', 'atac_mitochondrial_reads', 'atac_fragments', 'atac_TSS_fragments', 'atac_peak_region_fragments', 'atac_peak_region_cutsites', 'scDblFinder.class', 'scDblFinder.score', 'scDblFinder.weighted', 'scDblFinder.cxds_score', 'RNA_snn_res.0.8', 'seurat_clusters', 'sample', 'orig_sample_cluster', 'seurat_clusters_12162024', 'scDblFinder.score_12162024', 'RNA_snn_res.1.6', 'ident', 'batch', 'ID4', 'Brain_bank', 'seq_study_ID', 'Sex', 'Age', 'PMI', 'Diagnosis', 'Suicide', 'RIN', 'sample_barcode', 'harmony_rna_clusters_0.5', 'seurat_rna_clusters_0.5', 'harmony_rna_clusters_0.25', 'n_cells', 'predictions', 'cell_group', 'nucleosome_signal', 'nucleosome_percentile', 'TSS.enrichment', 'TSS.percentile', 'pct_reads_in_peaks', 'blacklist_fraction', 'atac_clusters_0.8

In [17]:
!pwd

/groups/mb928_gp/mp4486/Pyscenic


In [25]:
sub_1 = pd.read_csv('../palantir/astros_and_gcs_clusters_wnnsub5_0.5 (1).csv', index_col=0)
adata_subset.obs['astro_gc_wnn'] = sub_1.astype('category')

In [ ]:
sc.pl.embedding(adata_subset, basis='X_wnn.umap', color=['astro_gc_wnn'], frameon=False, legend_loc= 'on data')

In [28]:
# RANDOM SUBSET OF CELLS IF OBJECT IS TOO BIG

import numpy as np
import scanpy as sc
import pandas as pd

def get_adata_subset(adata, col, max_cells=2000):
    # Container for sampled indices
    sampled_indices = []

    # Loop through all unique cell types
    for ct in adata.obs[col].unique():
        ct_indices = adata.obs[adata.obs[col] == ct].index
        if len(ct_indices) > max_cells:
            sampled = np.random.choice(ct_indices, size=max_cells, replace=False)
        else:
            sampled = ct_indices
        sampled_indices.extend(sampled)
    
    # Subset the AnnData object
    adata_subset = adata[sampled_indices].copy()
    return adata_subset

get_adata_subset(adata_subset, 'astro_gc_wnn')

AnnData object with n_obs × n_vars = 35527 × 27510
    obs: 'orig.ident', 'nCount_ATAC', 'nFeature_ATAC', 'nCount_RNA', 'nFeature_RNA', 'mitoPercent', 'gex_barcode', 'gex_raw_reads', 'atac_barcode', 'atac_raw_reads', 'atac_unmapped_reads', 'atac_mitochondrial_reads', 'atac_fragments', 'atac_TSS_fragments', 'atac_peak_region_fragments', 'atac_peak_region_cutsites', 'scDblFinder.class', 'scDblFinder.score', 'scDblFinder.weighted', 'scDblFinder.cxds_score', 'RNA_snn_res.0.8', 'seurat_clusters', 'sample', 'orig_sample_cluster', 'seurat_clusters_12162024', 'scDblFinder.score_12162024', 'RNA_snn_res.1.6', 'ident', 'batch', 'ID4', 'Brain_bank', 'seq_study_ID', 'Sex', 'Age', 'PMI', 'Diagnosis', 'Suicide', 'RIN', 'sample_barcode', 'harmony_rna_clusters_0.5', 'seurat_rna_clusters_0.5', 'harmony_rna_clusters_0.25', 'n_cells', 'predictions', 'cell_group', 'nucleosome_signal', 'nucleosome_percentile', 'TSS.enrichment', 'TSS.percentile', 'pct_reads_in_peaks', 'blacklist_fraction', 'atac_clusters_0.8

## Module Creation from adjacencies
Regulons are derived from adjacencies based on three methods.

The **first** method to create the TF-modules is to select the best targets for each transcription factor:

1. Targets with importance > the 50th percentile.
2. Targets with importance > the 75th percentile
3. Targets with importance > the 90th percentile.


The **second** method is to select the top targets for a given TF:

1. Top 50 targets (targets with highest weight)

The alternative way to create the TF-modules is to select the best regulators for each gene (this is actually how GENIE3 internally works). Then, these targets can be assigned back to each TF to form the TF-modules. In this way we will create three more gene-sets:

1. Targets for which the TF is within its top 5 regulators
2. Targets for which the TF is within its top 10 regulators
3. Targets for which the TF is within its top 50 regulators

A distinction is made between modules which contain targets that are being activated and genes that are being repressed. Relationship between TF and its target, i.e. activator or repressor, is derived using the original expression profiles. The Pearson product-moment correlation coefficient is used to derive this information.

In addition, the transcription factor is added to the module and modules that have less than 20 genes are removed.

## Pruning modules based on Ranking Database and motif annotations

This step prunes the modules, only keeping the ones that are enriched with the motif that has a high ranking for those genes that is annotated with the given TF

In [ ]:
# List ranking databases and create FeatherRankingDatabase object
db_list = os.listdir('resources/ranking_dbs')
db_paths = ['resources/ranking_dbs/'+i for i in db_list]
def name(fname):
    return os.path.splitext(os.path.basename(fname))[0]
dbs = [RankingDatabase(fname=fname, name=name(fname)) for fname in db_paths]

In [28]:
# Run pipeline 50 times due to stochastic nature of algorithm

import warnings
import logging

# Suppress all warnings
warnings.filterwarnings('ignore')

# Suppress PySCENIC logs
logging.getLogger("pyscenic.prune").setLevel(logging.ERROR)
logging.getLogger("pyscenic.transform").setLevel(logging.ERROR)

for i in range(50):
    print(i)
    #subset = get_adata_subset(adata_subset, 'astro_gc_wnn', max_cells=2000)
    adata_mtx = adata_subset.X.toarray()
    exp_df = pd.DataFrame(data=adata_mtx, index=adata_subset.obs_names, columns=adata_subset.var_names)

    # Run GRN BOost2
    adjacencies = grnboost2(expression_data=adata_mtx, tf_names=list(overlapping_tfs), gene_names=adata_subset.var_names, client_or_address=client, verbose=True)
    
    print('Calculating modules')
    modules = list(modules_from_adjacencies(adjacencies, exp_df, rho_mask_dropouts=True, keep_only_activating=False, min_genes=10))
    pruned_modules = prune2df(dbs, modules, MOTIFS_PATH, client_or_address='custom_multiprocessing', num_workers=16, auc_threshold=0.01)

    regulons_test = df2regulons(pruned_modules)

    print(len(regulons_test))
    
    with open(f'neurogenesis/multi_runs2/regulons{i}.p', 'wb') as f:
        pickle.dump(regulons_test, f)

50
preparing dask client
parsing input
creating dask graph
32 partitions
computing dask graph
not shutting down client, client was created externally
finished
Calculating modules



2025-12-23 15:03:55,192 - pyscenic.utils - INFO - Calculating Pearson correlations.

2025-12-23 15:03:57,478 - pyscenic.utils - WARNING - Note on correlation calculation: the default behaviour for calculating the correlations has changed after pySCENIC verion 0.9.16. Previously, the default was to calculate the correlation between a TF and target gene using only cells with non-zero expression values (mask_dropouts=True). The current default is now to use all cells to match the behavior of the R verision of SCENIC. The original settings can be retained by setting 'rho_mask_dropouts=True' in the modules_from_adjacencies function, or '--mask_dropouts' from the CLI.
	Dropout masking is currently set to [True].

2025-12-23 15:04:18,219 - pyscenic.utils - INFO - Creating modules.


Create regulons from a dataframe of enriched features.
Additional columns saved: []
23
51
preparing dask client
parsing input
creating dask graph
32 partitions
computing dask graph
not shutting down client, client was created externally
finished
Calculating modules



2025-12-23 15:25:15,478 - pyscenic.utils - INFO - Calculating Pearson correlations.

2025-12-23 15:25:17,819 - pyscenic.utils - WARNING - Note on correlation calculation: the default behaviour for calculating the correlations has changed after pySCENIC verion 0.9.16. Previously, the default was to calculate the correlation between a TF and target gene using only cells with non-zero expression values (mask_dropouts=True). The current default is now to use all cells to match the behavior of the R verision of SCENIC. The original settings can be retained by setting 'rho_mask_dropouts=True' in the modules_from_adjacencies function, or '--mask_dropouts' from the CLI.
	Dropout masking is currently set to [True].

2025-12-23 15:25:38,708 - pyscenic.utils - INFO - Creating modules.


Create regulons from a dataframe of enriched features.
Additional columns saved: []
33
52
preparing dask client
parsing input
creating dask graph
32 partitions
computing dask graph
not shutting down client, client was created externally
finished
Calculating modules


...

Create regulons from a dataframe of enriched features.
Additional columns saved: []
30
98
preparing dask client
parsing input
creating dask graph
32 partitions
computing dask graph
not shutting down client, client was created externally
finished
Calculating modules



2025-12-24 08:30:31,798 - pyscenic.utils - INFO - Calculating Pearson correlations.

2025-12-24 08:30:34,130 - pyscenic.utils - WARNING - Note on correlation calculation: the default behaviour for calculating the correlations has changed after pySCENIC verion 0.9.16. Previously, the default was to calculate the correlation between a TF and target gene using only cells with non-zero expression values (mask_dropouts=True). The current default is now to use all cells to match the behavior of the R verision of SCENIC. The original settings can be retained by setting 'rho_mask_dropouts=True' in the modules_from_adjacencies function, or '--mask_dropouts' from the CLI.
	Dropout masking is currently set to [True].

2025-12-24 08:31:02,930 - pyscenic.utils - INFO - Creating modules.


Create regulons from a dataframe of enriched features.
Additional columns saved: []
26
99
preparing dask client
parsing input
creating dask graph
32 partitions
computing dask graph
not shutting down client, client was created externally
finished
Calculating modules



2025-12-24 08:52:24,430 - pyscenic.utils - INFO - Calculating Pearson correlations.

2025-12-24 08:52:26,743 - pyscenic.utils - WARNING - Note on correlation calculation: the default behaviour for calculating the correlations has changed after pySCENIC verion 0.9.16. Previously, the default was to calculate the correlation between a TF and target gene using only cells with non-zero expression values (mask_dropouts=True). The current default is now to use all cells to match the behavior of the R verision of SCENIC. The original settings can be retained by setting 'rho_mask_dropouts=True' in the modules_from_adjacencies function, or '--mask_dropouts' from the CLI.
	Dropout masking is currently set to [True].

2025-12-24 08:52:55,153 - pyscenic.utils - INFO - Creating modules.


Create regulons from a dataframe of enriched features.
Additional columns saved: []
27
